In [86]:
import pandas as pd
import numpy as np

In [87]:
cnv_tsv = '/s/project/mll/rawdata/res_from_mll/cnv_calls.txt'

gene_coords = '/s/project/mll/sergey/effect_prediction/promoter_mutations/genes_GRCh37.tsv.gz' #biomart, HGNC

#gene_coords = '/s/project/mll/sergey/effect_prediction/promoter_mutations/genes_GRCh37/genes_GRCh37.bed'

In [88]:
cnv_df = pd.read_csv(cnv_tsv, sep='\t',
                    usecols=[0,1,2,5,6], names=['chrom','start','end','call','array_id'], skiprows=1)

In [89]:
cnv_df = cnv_df[~cnv_df.chrom.isna()]

In [90]:
#gene_df = pd.read_csv(gene_coords, sep='\t', comment='#',
#                      names=['chrom','start','end','geneName'])

In [91]:
gene_df = pd.read_csv(gene_coords, sep='\t', comment='#',usecols=[0,2,3,8],
                      names=['chrom','start','end','geneHGNC']).drop_duplicates()

gene_df.chrom = gene_df.chrom.apply(lambda x: x.replace('chr',''))

#tx_start = gene_df.groupby(['geneHGNC','chrom']).start.min()
#tx_end = gene_df.groupby(['geneHGNC','chrom']).end.max()

#gene_df = pd.concat([tx_start,tx_end],axis=1).reset_index()

gene_df.sort_values(by = ['chrom','start', 'end'], inplace=True)

In [92]:
def getOverlap(a, b):
    '''
    Returns the amount of overlap between two intervals
    '''
    #print(a,b)
    return max(0, min(a[1], b[1]) - max(a[0], b[0]))

def get_nonoverlap_ranges(df):
    intervals = df[['start','end']].values.tolist()
    merged_intervals = []
    current_interval = intervals[0]
    for idx in range(1,len(intervals)):
        if getOverlap(current_interval,intervals[idx]):
            current_interval = [current_interval[0],intervals[idx][1]]
        else:
            merged_intervals.append(current_interval)
            current_interval = intervals[idx]
    merged_intervals.append(current_interval)
    return pd.DataFrame(merged_intervals, columns=['start','end'])

In [93]:
gene_df = gene_df.groupby(['geneHGNC','chrom']).apply(lambda x: get_nonoverlap_ranges(x)).reset_index().drop(columns='level_2')

In [94]:
cnv_df = cnv_df[cnv_df.call!='0'] 

In [95]:
intervals_df = cnv_df[['chrom','start','end']].drop_duplicates()

intervals_df['geneHGNC'] = None
intervals_df['geneHGNC'] = intervals_df['geneHGNC'].astype(object)

In [96]:
MIN_OVERLAP = 1 #minimum overlap between CNV and gene to accept CNV

In [97]:
#loop over copy number intervals and check if they overlap with genes

for chrom in cnv_df.chrom.unique():
    
    print(chrom)
    
    #select given chromosome
    intervals_chrom_df = intervals_df[intervals_df.chrom==chrom]
    gene_chrom_df = gene_df[gene_df.chrom==chrom] #gene coordinates in given chromosome
    
    for idx, row in intervals_chrom_df.iterrows():
        int_start, int_end = row.start, row.end
        #interval end should be larger than the gene start
        max_row_idx = np.searchsorted(gene_chrom_df.start, int_end) #gene_df should already be sorted by chromosome and by start
        df = gene_chrom_df.iloc[:max_row_idx].sort_values(by='end')
        #interval start should be smaller than the gene end
        min_row_idx = np.searchsorted(df.end, int_start)
        df = df.iloc[min_row_idx:] #now all intervals remaining in df should overlap
        
        sufficient_overlap = df.apply(lambda x: getOverlap((x.start,x.end),(int_start, int_end)), axis=1)>MIN_OVERLAP
        
        intervals_df.at[idx,'geneHGNC'] = df.loc[sufficient_overlap].geneHGNC.values.tolist() #add the list of genes for given interval

        #if len(df):
        #    assert isOverlap((df.iloc[0].start,df.iloc[0].end),(int_start, int_end))
        #    assert isOverlap((df.iloc[-1].start,df.iloc[-1].end),(int_start, int_end))       

1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
17
18
19
20
21
22
X
10
Y


In [98]:
intervals_df.geneHGNC = intervals_df.geneHGNC.apply(lambda x:x[0] if type(x)==list and len(x)==1 else x) #list to string if only 1 item

In [99]:
cnv_df = cnv_df.merge(intervals_df, how='left')

In [100]:
cnv_df = cnv_df.explode('geneHGNC')

In [101]:
cnv_df = cnv_df[~cnv_df.geneHGNC.isna()].drop_duplicates()

In [102]:
matching_genes = pd.read_csv('/s/project/mll/sergey/effect_prediction/promoter_mutations/ensemble_to_HGNC_GRCh38.tsv.gz', sep='\t', 
                      header=None, names=['geneName', 'geneHGNC'], usecols=[0,1], skiprows=1)

matching_genes = matching_genes[~matching_genes.geneHGNC.isna()]

In [103]:
cnv_df = pd.merge(cnv_df,matching_genes, how='left')

In [104]:
cnv_df

,chrom,start,end,call,array_id,geneHGNC,geneName
0,1,142540001,142838000,+,MLL_16348,ANKRD20A12P,NaN
1,1,149037001,149433000,-,MLL_16348,FCGR1CP,ENSG00000265531
2,1,149037001,149433000,-,MLL_16348,H2BP2,ENSG00000234571
3,1,149037001,149433000,-,MLL_16348,H3-7,ENSG00000273213
4,1,196730001,196805000,-,MLL_16348,CFHR3,ENSG00000116785
...,...,...,...,...,...,...,...
5231924,20,25655001,26318000,-,MLL_62407,MIR663A,ENSG00000284419
5231925,20,25655001,26318000,-,MLL_62407,MIR663AHG,ENSG00000227195
5231926,21,40558001,40570000,+,MLL_62407,BRWD1,ENSG00000185658
5231927,21,43350001,43354000,-,MLL_62407,C2CD2,ENSG00000157617


In [105]:
#take a subsample to check overlaps

short_cnv = cnv_df[['chrom','start','end','geneHGNC']].drop_duplicates().sample(n=10000)

In [106]:
df = gene_df.set_index(['geneHGNC','chrom'])

In [107]:
#check overlaps

def get_max_overlap(interval1, interval_list):
    max_ovp = -1
    for interval2 in interval_list:
        max_ovp = max(max_ovp,max(0, min(interval1[1], interval1[1]) - max(interval2[0], interval2[0])))
    return max_ovp

overlaps=short_cnv.groupby('geneHGNC').apply(lambda x: 
                                 x.apply(lambda y: get_max_overlap((y.start,y.end),
                                                             df.loc[(x.name,y.chrom)].values.tolist())>MIN_OVERLAP,axis=1))


In [108]:
overlaps.mean()

1.0

In [109]:
cnv_df.to_csv('/s/project/mll/sergey/MLL_data/processed/cnv_mll_NoOverlapNoMaxlen.tsv', sep='\t', index=None)